# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [28]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(99, 'malE', 0.43)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(10, 'mAle', 0.30)
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok
Trying:
    find_hospital(50, 'F', '0.7')
Expecting:
    'Emory Dunwoody Medical Center'
ok


In [29]:
def priest(sex, age, rr, o, h, sb, t, alert, aso, ps):

  """((str, int, int, int, int, float, float, str, str, str)) -> (float)

    Using input variables in this function calculates the patient’s risk percentage from
    different clinical measurements such as sex, age, respiratory rate, oxygen saturation, pulse rate,
    systolic blood pressure, temperature, consciousness level, type of inspired oxygen, and performance status.

    - sex (str): Biological sex (male, female).
    - age (int): Age in years.
    - rr (int): Respiratory rate (breaths per minute).
    - o (float): Patient's oxygen saturation level (0 to 1).
    - h (int): The heart rate (beats per minute).
    - sb (int): The systolic blood pressure (mmHg).
    - t (float): Temperature (Celsius).
    - alert (str): Alertness status ("alert" or "confused or not alert").
    - aso (str): Type of oxygen intake ("air" or "supplemental oxygen").
    - ps (str): Physical performance status, reflecting their activity level:
                         Can be one of the following:
                         - "unrestricted normal activity"
                         - "limited strenuous activity, can do light activity"
                         - "limited activity, can self-care"
                         - "limited self-care"
                         - "bed/chair bound, no self-care"

    Returns:
    returning_p (float): Risk percentage (from 0 to 1) that estimates the patient's clinical risk based on the parameters. We will convert the total points into a risk percentage based on
    predefined thresholds.

    None: Returns None if any input value or data type is incorrect.
    >>> priest('MalE', 60, 11, 0.94, 92, 90, 31.2,'Confused or Not alert', 'Air', 'limited strenuous activity, can do light activity')
    0.55

    >>> priest('MAle', 90, 90, 0.90, 40, 40, 30.0, 'Alert', 'Air', 'limited strenuous activity, can do light activity')
    0.59


    """
    # Initialize for calculating the risk points and risk percentage
  add_p = 0
  returning_p = 0

  # Convert input str variables to lowercase
  sex = sex.lower()
  alert = alert.lower()
  aso = aso.lower()
  ps = ps.lower()

  # Validate data types and input ranges
  if type(age) != int:
      return None
  if type(rr) != int:
      return None
  if type(h) != int:
      return None
  if type(sb) != int:
      return None
  if type(o) != float:

      return None
  if type(t) != float:
      return None
  if ps not in ["unrestricted normal activity", "limited strenuous activity, can do light activity", "limited activity, can self-care", "limited self-care", "bed/chair bound, no self-care"]:
      return None
  if sex not in ["male", "female"]:
      return None
  if aso not in ["air", "supplemental oxygen"]:
      return None
  if alert not in ["alert", "confused or not alert"]:
      return None
      # Points for sex
  add_p += 1 if sex == "male" else 0

  # Points for respiratory rate
  if rr < 9:
      add_p += 3
  elif 9 <= rr <= 11:
      add_p += 1
  elif 21 <= rr <= 24:
      add_p += 2
  elif rr > 24:
      add_p += 3

  # Points for oxygen saturation
  if o > 0.95:
      add_p += 0
  elif 0.94 <= o <= 0.95:
      add_p += 1
  elif 0.92 <= o <= 0.93:
      add_p += 2
  elif o < 0.92:
      add_p += 3

  # Points for age
  if age > 80:
      add_p += 4
  elif 66 <= age <= 80:
      add_p += 3
  elif 50 <= age <= 65:
      add_p += 2

  # Points for heart rate
  if h < 41 or h > 130:
      add_p += 3
  elif 41 <= h <= 50 or 91 <= h <= 110:
      add_p += 1
  elif 111 <= h <= 130:
      add_p += 2

 # Points for temperature C
  if t < 35.1:
      add_p += 3
  elif 35.1 <= t <= 36.0 or 38.1 <= t <= 39.0:
      add_p += 1

  elif 36.1 <= t <= 38.0:
      add_p += 0
  else:
      add_p += 2

  # Points for systolic BP
  if sb < 91:
      add_p += 3
  elif 91 <= sb <= 100:
      add_p += 2
  elif 101 <= sb <= 110:
      add_p += 1
  elif sb > 219:
      add_p += 3


  # Points for alertness
  if alert == "confused or not alert":
      add_p += 3
  else:
      add_p += 0

  # Points for inspired oxygen
  add_p += 2 if aso == "supplemental oxygen" else 0

  # Points for performance status
  if ps == "bed/chair bound, no self-care":
      add_p += 4
  elif ps == "limited strenuous activity, can do light activity":
      add_p += 1
  elif ps == "limited activity, can self-care":
      add_p += 2

  elif ps == "limited self-care":
      add_p += 3
  elif ps == "unrestricted normal activity":
      add_p += 0

  # Points to risk percentage
  if add_p >= 0 and add_p <= 1:
      returning_p += 0.01

  elif 2 <= add_p <= 3:
      returning_p += 0.02

  elif add_p == 4:
      returning_p += 0.03

  elif add_p == 5:
      returning_p += 0.09

  elif add_p == 6:
      returning_p += 0.15

  elif add_p == 7:
      returning_p += 0.18

  elif add_p == 8:
      returning_p += 0.22

  elif add_p == 9:
      returning_p += 0.26

  elif add_p == 10:
      returning_p += 0.29

  elif add_p == 11:
      returning_p += 0.34

  elif add_p == 12:
      returning_p += 0.38

  elif add_p == 13:
      returning_p += 0.46

  elif add_p == 14:
      returning_p += 0.47

  elif add_p == 15:
      returning_p += 0.49

  elif add_p == 16:
      returning_p += 0.55

  elif 17 <= add_p <= 25:
      returning_p += 0.59
  else:
      returning_p += 0.99

# Final risk percentage return
  return returning_p

In [30]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('MalE', 60, 11, 0.94, 92, 90, 31.2,'Confused or Not alert', 'Air', 'limited strenuous activity, can do light activity')
Expecting:
    0.55
ok
Trying:
    priest('MAle', 90, 90, 0.90, 40, 40, 30.0, 'Alert', 'Air', 'limited strenuous activity, can do light activity')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [31]:
import requests

In [32]:
def find_hospital(age, sex, p):
  """(int, str, float) -> (str)
  Accessing a web service to obtain information about hospitals and return the hospital name.

  >>> find_hospital(99, 'malE', 0.43)
  'Wesley Woods Geriatric Hospital'

  >>> find_hospital(10, 'mAle', 0.30)
  'Childrens Healthcare of Atlanta at Scottish Rite'

  >>> find_hospital(50, 'F', '0.7')
  'Emory Dunwoody Medical Center'

  """
  # API URL with parameters
  url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={p}'

  # GET request
  rizz = requests.get(url)

  # Parse the JSON
  rizz_data = rizz.json()

  # Extract name
  nam = rizz_data.get('hospital')

  # Return name
  if rizz_data:
    return nam
  else:
    return None

In [33]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(99, 'malE', 0.43)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(10, 'mAle', 0.30)
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok
Trying:
    find_hospital(50, 'F', '0.7')
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [34]:
import requests
import json

In [35]:
def get_address(name):
  """(str) -> (str)
   Utilizing the retrieved hospital information to acquire its address.

   >>> get_address('SHEPHERD CENTER')
   '2020 PEACHTREE ROAD NW'

   >>> get_address('BUCKHEAD AMBULATORY SURGICAL CENTER')
   '2061 PEACHTREE RD NW, UNIT 300'

  """

  # Loading json
  drive = 'https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-'

  load_json = requests.get(drive)
  rj = load_json.json()

  # Convert and check
  name = name.upper()

  if name in rj:
    return rj[name]['ADDRESS']

  else:
    return None



In [36]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('SHEPHERD CENTER')
Expecting:
    '2020 PEACHTREE ROAD NW'
ok
Trying:
    get_address('BUCKHEAD AMBULATORY SURGICAL CENTER')
Expecting:
    '2061 PEACHTREE RD NW, UNIT 300'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [37]:
import csv
import requests

In [38]:
def process_people(drive):
    """(str) -> (list)

    Get info from URL and return a list of dictionaries containing
    patient information, including health and hospital details.

    If any invalid inputs are found, the function returns None.

    drive : Drive URL
    """

    # Download
    rizz = requests.get(drive)
    # PSV
    with open('people.psv', 'w') as f:
        f.write(rizz.text)

    # Open and read
    with open('people.psv', 'r') as file:
      # Read with delimiter
        ctv = csv.reader(file, delimiter='|')

        r = next(ctv)  # get  header

        retur = []  # List to hold dictionaries

        dt = {}
        for row in ctv:
            # Extract values and put them in there correct datatypes.

            p, sex, age, b, oxs, htr, stb, tc, alt, otwo, sts = (
                    row[r.index('patient')], row[r.index('sex')], int(row[r.index('age')]),
                    int(row[r.index('breath')]), float(row[r.index('o2 sat')]),
                    int(row[r.index('heart')]), int(row[r.index('systolic bp')]),
                    float(row[r.index('temp')]), row[r.index('alertness')],
                    row[r.index('inspired')], row[r.index('status')]
                    )

            # Get risk using the priest function
            risk = priest(sex, age, b, oxs, htr, stb, tc, alt, otwo, sts)

            # Use functions to get hospital details
            name = find_hospital(age, sex, risk)

            if name is None:
                return None

            gadd = get_address(str(name))

            if gadd is None:
                return None

            # Make dictionary
            p_details = {
                p: [
                    sex, age, float(b), float(oxs),
                    float(htr), float(stb),
                    float(tc), alt, otwo,
                    sts, risk, name, gadd
                ]
            }
            dt.update(p_details)  # Put them in list
        retur.append(dt)

    return retur  # Return list of dictionaries


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [39]:
import requests
import json

In [40]:
# Loading the first URL to fetch JSON data
drive1 = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'
drive2 = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'

In [27]:
# Final test
retur = []

# Fetch and save JSON data from the first drive URL
rdj = requests.get(drive1)
rdj = rdj.json()

# Save the JSON data
with open('data.json', 'w') as i:
    json.dump(rdj, i)

retur.append(rdj)  # Append saved JSON data

# Check my work
if process_people(drive2) == retur:
    print('Both of them are equal')
else:
    print('Both of them are not Equal')

Both of them are equal


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---